# <center>CO2 LSTM Forecasting</center>

> Authors:
> - D11202805 - Ian Joseph Chandra
> - M11002818 - Wilfrid Azariah

# Step 1: Import Libraries

In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from datetime import timedelta

# Load data
df = pd.read_csv('./data/emissions.csv')

df


,year,state-name,sector-name,fuel-name,value
0,1970,Alabama,Industrial carbon dioxide emissions,Coal,26.721507
1,1970,Alabama,Industrial carbon dioxide emissions,Petroleum,3.577779
2,1970,Alabama,Industrial carbon dioxide emissions,Natural Gas,8.944097
3,1970,Alabama,Industrial carbon dioxide emissions,All Fuels,39.243383
4,1970,Alabama,Total carbon dioxide emissions from all sectors,All Fuels,102.646851
...,...,...,...,...,...
59896,2021,Wyoming,Commercial carbon dioxide emissions,Coal,0.012374
59897,2021,Wyoming,Residential carbon dioxide emissions,All Fuels,0.937989
59898,2021,Wyoming,Residential carbon dioxide emissions,Natural Gas,0.717777
59899,2021,Wyoming,Residential carbon dioxide emissions,Petroleum,0.220212


# Step 2: Data Preprocessing

In [50]:
# Remove "-name" from column names
df.columns = df.columns.str.replace('-name', '')

# Remove "carbon dioxide emissions" from 'sector' column
df['sector'] = df['sector'].str.replace('carbon dioxide emissions', '')

# Remove rows where 'fuel' column contains "All Fuels"
df = df[~df['fuel'].str.contains('All Fuels')]

# Remove rows where 'sector' column contains "Total"
df = df[~df['sector'].str.contains('Total')]

df


,year,state,sector,fuel,value
0,1970,Alabama,Industrial,Coal,26.721507
1,1970,Alabama,Industrial,Petroleum,3.577779
2,1970,Alabama,Industrial,Natural Gas,8.944097
8,1970,Alabama,Residential,Coal,0.163635
9,1970,Alabama,Residential,Petroleum,1.123947
...,...,...,...,...,...
59894,2021,Wyoming,Commercial,Natural Gas,0.686983
59895,2021,Wyoming,Commercial,Petroleum,0.216315
59896,2021,Wyoming,Commercial,Coal,0.012374
59898,2021,Wyoming,Residential,Natural Gas,0.717777


In [51]:
# Drop the 'state' column
df = df.drop('state', axis=1)

# Group by 'year', 'sector', and 'fuel' and sum 'value'
df_grouped = df.groupby(['year', 'sector', 'fuel'], as_index=False)['value'].sum()

df_grouped

,year,sector,fuel,value
0,1970,Commercial,Coal,31.306261
1,1970,Commercial,Natural Gas,260.289232
2,1970,Commercial,Petroleum,233.356940
3,1970,Electric Power,Coal,1374.959395
4,1970,Electric Power,Natural Gas,429.208459
...,...,...,...,...
717,2021,Industrial,Petroleum,698.180684
718,2021,Residential,Natural Gas,520.017628
719,2021,Residential,Petroleum,132.467958
720,2021,Transportation,Natural Gas,129.829709
